In [479]:
import pandas as pd
import altair as alt

In [480]:
dados_federais_campina = pd.read_csv("data\\dados_federais_obitos_cg.csv", encoding='utf-8', error_bad_lines=False)
dados_painelfederal_campina = pd.read_csv("data\\dados_painel-federal_obitos_cg.csv", encoding='UTF-8', error_bad_lines=False)
dados_municipais_campina = pd.read_csv("data\\dados_municipais_cg.csv", encoding='utf-8', index_col='data')

### Função que exclui os elementos (pelo index) do DataFrame que não estão em outro DataFrame

In [481]:
def exclui_nao_contidos(df_conjunto, df_subconjunto):
    for i in df_conjunto.index:
        if i not in df_subconjunto.index:
            df_conjunto = df_conjunto.drop(i)
    
    return df_conjunto

## Processando os dados sobre Obitos Confirmados por Dia (Painel Federal)

In [482]:
# Criando um data com a coluna 'data' e a coluna 'obitosNovos'
pf_obitosPorDia = dados_painelfederal_campina[['obitosNovos', 'data']]


In [483]:
# Criando um DataFrame com os dados para o altair
pf_obitosPorDia = pd.DataFrame({
    'data': pf_obitosPorDia['data'].tolist(),
    'obitos_confirmados': pf_obitosPorDia['obitosNovos'].tolist()
})

# Criando uma nova coluna no DataFrame referente a média movel.
pf_obitosPorDia['media_movel'] = pf_obitosPorDia.obitos_confirmados.rolling(7).mean()

# Setando a coluna 'data' como índice do DataFrame 
pf_obitosPorDia.set_index('data', inplace=True)


## Processando os dados sobre Obitos Confirmados por Dia (Dados Federais)

In [484]:
# Criando um data com a coluna 'dataNotificacao' e a coluna 'saidaConfirmadaObitos'
df_obitosPorDia = dados_federais_campina[['dataNotificacao', 'saidaConfirmadaObitos']]

#### OBS: Os dados federais estão descritos pela data e cada data diluida pelo estabelecimento de saúde ao qual foi notificado.

In [485]:
# Agrupando as informações de 'saidaConfirmadaObitos' pela data de notificação
df_obitosPorDia = df_obitosPorDia.groupby('dataNotificacao')
df_obitosPorDia = df_obitosPorDia['saidaConfirmadaObitos'].apply(list)
df_obitosPorDia = df_obitosPorDia.reset_index()

# Somando os valores que foram agrupados pela data
for i in df_obitosPorDia.index:
    df_obitosPorDia.loc[df_obitosPorDia.index == i, 'saidaConfirmadaObitos'] = sum(df_obitosPorDia.iloc[i]['saidaConfirmadaObitos'])


In [486]:
# Criando um DataFrame com os dados para o altair
df_obitosPorDia = pd.DataFrame({
    'data': df_obitosPorDia['dataNotificacao'].tolist(),
    'obitos_confirmados': df_obitosPorDia['saidaConfirmadaObitos'].tolist()
})

# Criando uma nova coluna no DataFrame referente a média movel.
df_obitosPorDia['media_movel'] = df_obitosPorDia.obitos_confirmados.rolling(7).mean()

# Setando a coluna 'data' como índice do DataFrame 
df_obitosPorDia.set_index('data', inplace=True)

### Excluindo as datas não contidas nos DataFrames (Painel Federal e Federal)

In [487]:
# Excluindo as datas que estão nos dados do painel federal mas não estão nos dados federais
pf_obitosPorDia = exclui_nao_contidos(pf_obitosPorDia, df_obitosPorDia)

# Excluindo as datas que estão nos dados federais mas não estão nos dados do painel federal
df_obitosPorDia = exclui_nao_contidos(df_obitosPorDia, pf_obitosPorDia)

# Processando os sobre o Numero de Obitos por Dia (Dados Municipais)

In [488]:
# Criando um data com a coluna 'data' e a coluna 'obitosNovos'
dm_obitosPorDia = dados_municipais_campina['obitosPorDia']

In [489]:
# Criando um DataFrame com os dados para o altair
dm_obitosPorDia = pd.DataFrame({
    'data': dm_obitosPorDia.index.tolist(),
    'obitos_confirmados': dm_obitosPorDia.values.tolist()
})

# Criando uma nova coluna no DataFrame referente a média movel.
dm_obitosPorDia['media_movel'] = dm_obitosPorDia.obitos_confirmados.rolling(7).mean()

# Setando a coluna 'data' como índice do DataFrame 
dm_obitosPorDia.set_index('data', inplace=True)


### Excluindo as datas não contidas nos DataFrames (Painel Federal e Municipal)

In [490]:
# Excluindo as datas que estão nos dados federais mas não estão nos dados municipais
pf_obitosPorDia = exclui_nao_contidos(pf_obitosPorDia, dm_obitosPorDia)

# Excluindo as datas que estão nos dados municipais mas não estão nos dados federais
df_obitosPorDia = exclui_nao_contidos(df_obitosPorDia, dm_obitosPorDia)

### Transformando os DataFrame do formato wide para o formato long

In [491]:
# Transformando os DataFrames sobre obitos confirmados
obitosPorDia = pd.DataFrame({
  'data': pf_obitosPorDia.index.tolist(),
  'Painel Federal': pf_obitosPorDia['obitos_confirmados'].tolist(),
  'Federal': df_obitosPorDia['obitos_confirmados'].tolist(),
  'Municipal': dm_obitosPorDia['obitos_confirmados'].tolist()
}).melt(id_vars=['data'], value_vars=['Painel Federal', 'Federal', 'Municipal'],
        var_name='fonte_dados', value_name='aplic_por_dia')

In [492]:
# Transformando os DataFrames sobre a média movel dos obitos confirmados
obitosPorDiaMedMovel = pd.DataFrame({
  'data': pf_obitosPorDia.index.tolist(),
  'Painel Federal': pf_obitosPorDia['media_movel'].tolist(),
  'Federal': df_obitosPorDia['media_movel'].tolist(),
  'Municipal': dm_obitosPorDia['media_movel'].tolist()
}).melt(id_vars=['data'], value_vars=['Painel Federal', 'Federal', 'Municipal'],
        var_name='fonte_dados', value_name='aplic_por_dia')

# Comparando os dados do Painel Federal, Federais e Municipais

In [493]:
alt.Chart(obitosPorDia).mark_line().encode(
    alt.X('data:T', title='Data'),
    alt.Y('aplic_por_dia:Q', title='Nº de Obitos'),
    alt.Color('fonte_dados:N', title='Base dos Dados')
).properties(
    width=700,
    title='Quantidade de Obitos Confirmados por Dia'
)

alt.Chart(...)

### Média Movel (7 dias) dos Obitos Confirmados

In [494]:
alt.Chart(obitosPorDiaMedMovel).mark_line(point=True).encode(
    alt.X('data:T', title='Data'),
    alt.Y('aplic_por_dia:Q', title='Nº de Obitos'),
    alt.Color('fonte_dados:N', title='Base dos Dados')
).properties(
    width=750,
    title='Média Movel de Obitos Confirmados por Dia (Comparação)'
)

alt.Chart(...)

### Total de Obitos Confirmados

In [495]:
pf_obitosTotais = pf_obitosPorDia['obitos_confirmados'].sum()
df_obitosTotais = df_obitosPorDia['obitos_confirmados'].sum()
dm_obitosTotais = dm_obitosPorDia['obitos_confirmados'].sum()

In [496]:
totalObitos = pd.DataFrame({
    'base': ['Painel Federal', 'Federal', 'Municipal'],
    'total': [pf_obitosTotais, df_obitosTotais, dm_obitosTotais]
})

In [497]:
alt.Chart(totalObitos).mark_bar(size=30).encode(
    alt.X('base:N', title='Base dos Dados'),
    alt.Y('total:Q', title='Nº Total de Obitos Confirmados'),
    alt.Color('base:N', title='Base de Dados')
).properties(
    width=150,
    title='Total de Obitos Confirmados (Comparação)'
)

alt.Chart(...)